In [ ]:
def bsm_call_value (S0, K, T, r, sigma):
  '''
  Parameters:
  S0 : float - initial stock/index level 
  K : float - strike price 
  T : float - maturity date (in year fractions) 
  r : float - constant risk-free short rate 
  sigma : float - volatility factor in diffusion term 

  Returns:
  value : float - present value of the European call option
  '''

  from math import log, sqrt, exp
  from scipy import stats
  
  S0 = float(S0)
  d1 = (log(S0/K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
  d2 = (log(S0 / K) + (r - 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
  value = (S0 * stats.norm.cdf(d1, 0.0, 1.0) - K * exp (-r * T) * stats.norm.cdf(d2, 0.0, 1.0))
  return value

In [ ]:
def bsm_vega(S0, K, T, r, sigma): 
  '''
  Parameters:
    S0 : float - initial stock/index level 
    K : float - strike price 
    T : float maturity date (in year fractions) 
    r : float - constant risk-free short rate 
    sigma : float - volatility factor in diffusion term

  Returns:
    vega : float - partial derivative of BSM formula with respect to sigma, i.e. Vega
  ''' 

  from math import log, sqrt 
  from scipy import stats 
  S0 = float(S0) 
  d1 = (log(S0/K) + (r + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
  vega = S0 * stats.norm.cdf(d1, 0.0, 1.0) * sqrt(T)
  return vega

In [ ]:
def bsm_call_imp_vol(S0, K, T, r, C0, sigma_est, it = 100): 
  ''' 
  Parameters:
    S0 : float - initial stock/index level 
    K : float - strike price 
    T : float - maturity date (in year fractions) 
    r : float - constant risk-free short rate 
    sigma_est : float estimate of impl. volatility 
    it : integer number of iterations 

  Returns:
    simga_est : float - numerically estimated implied volatility
  '''

  for i in range ( it ): 
    sigma_est -= ((bsm_call_value(S0, K, T, r, sigma_est) - C0) / bsm_vega(S0, K, T, r, sigma_est))
  return sigma_est

In [ ]:
from dao import InvestDao
dao = InvestDao(host="172.17.0.1", port=3306, user="user", passwd="passwd", db="db")

In [ ]:
r = dao.sql("select rate from rates where dtyymmdd='2017-06-02' and symbol='WIBOR6M'")[0]
rate = float(r[0])/100.0

In [ ]:
(ttm, strike, close) = dao.sql("select ttm, strike, close from v_option_quotes where ticker='OW20C181800' and dtyymmdd='2017-06-02' and type='C'")[0]

In [ ]:
(initial, volatility) = dao.sql("select close, std_close from v_stock_stats where dtyymmdd = '2017-06-02'")[0]

In [ ]:
bsm_call_imp_vol(float(initial), float(strike), float(ttm), float(rate), float(close), float(volatility))